In [57]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo_request al host pasado por parámetro. La función devuelve un diccionario que contiene la siguiente información:

* dst: Host contactado
* response: True si se obtuvo respuesta; False en caso contrario
* rtt_us: RTT en microsegundos (None si no se obtuvo respuesta)
* len: Longitud del paquete recibido (None si no se obtuvo respuesta)
* ttl: Time to live del paquete recibido (None si no se obtuvo respuesta)


In [58]:
def ping(host):

    data = dict()
    data["dst"] = host

    print(f"Pinging {host}...")
    packet = IP(dst=host)/ICMP(type=8)
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    inicio = time.time()
    if response:
        fin = time.time()
        rtt_mcs = (fin - inicio) * 1000000
        print(f"Respuesta recibida de {host}")
        
        data["response"] = True
        data["rtt_us"] = rtt_mcs
        data["len"] = response.len
        data["ttl"] = response.ttl

    else:
        print(f"No se obtuvo respuesta de {host}")
        data["response"] = False
        data["rtt_us"] = None
        data["len"] = None
        data["ttl"] = None
    

    return data

Probamos contactar a utdt.edu.ar

In [59]:
ping("utdt.edu.ar")

Pinging utdt.edu.ar...
Respuesta recibida de utdt.edu.ar


{'dst': 'utdt.edu.ar',
 'response': True,
 'rtt_us': 1.430511474609375,
 'len': 28,
 'ttl': 56}

Ahora, creamos una función que recibe un set de destinos a contactar. Devuelve un diccionario que contiene:
* Cantidad de paquetes enviados
* Cantidad de paquetes recibidos
* Cantidad de paquetes perdidos
* Porcentaje de paquetes perdidos
* RTT promedio
* RTT mínimo
* RTT máximo
* Desvío estándar del RTT promedio A QUE SE REFIERE?


In [60]:
def ping_multiple_hosts(hosts):

    res = dict()

    sent_packets = len(hosts)
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    
    for host in hosts:
        resultado = ping(host)
        print(resultado)

        if resultado["response"]: # Si se recibió respuesta
            received_packets += 1
            total_rtt += resultado["rtt_us"]
            min_rtt = min(min_rtt, resultado["rtt_us"])
            max_rtt = max(max_rtt, resultado["rtt_us"])
        else:
            lost_packets += 1
    
    if received_packets > 0:
        average_rtt = total_rtt / received_packets

    res["hosts"] = hosts
    res["sent_packets"] = sent_packets
    res["received_packets"] = received_packets
    res["lost_packets"] = lost_packets
    res["loss_percentage"] = (lost_packets / sent_packets) 
    res["total_rtt"] = total_rtt
    res["min_rtt"] = min_rtt if min_rtt != float('inf') else None
    res["max_rtt"] = max_rtt if max_rtt != float('-inf') else None
    res["average_rtt"] = average_rtt if received_packets > 0 else None
    
    return res

Contactamos a 5 universidades de continentes distintos

In [61]:

destinos = {"mit.edu", "ie.edu", "sydney.edu.au", "cuhk.edu.cn", "alexu.edu.eg"}

ping_multiple_hosts(destinos)

Pinging sydney.edu.au...
Respuesta recibida de sydney.edu.au
{'dst': 'sydney.edu.au', 'response': True, 'rtt_us': 5.7220458984375, 'len': 28, 'ttl': 110}
Pinging cuhk.edu.cn...
Respuesta recibida de cuhk.edu.cn
{'dst': 'cuhk.edu.cn', 'response': True, 'rtt_us': 3.814697265625, 'len': 28, 'ttl': 112}
Pinging alexu.edu.eg...
Respuesta recibida de alexu.edu.eg
{'dst': 'alexu.edu.eg', 'response': True, 'rtt_us': 0.7152557373046875, 'len': 28, 'ttl': 45}
Pinging ie.edu...
Respuesta recibida de ie.edu
{'dst': 'ie.edu', 'response': True, 'rtt_us': 0.7152557373046875, 'len': 28, 'ttl': 248}
Pinging mit.edu...
Respuesta recibida de mit.edu
{'dst': 'mit.edu', 'response': True, 'rtt_us': 0.7152557373046875, 'len': 28, 'ttl': 53}


{'hosts': {'alexu.edu.eg',
  'cuhk.edu.cn',
  'ie.edu',
  'mit.edu',
  'sydney.edu.au'},
 'sent_packets': 5,
 'received_packets': 5,
 'lost_packets': 0,
 'loss_percentage': 0.0,
 'total_rtt': 11.682510375976562,
 'min_rtt': 0.7152557373046875,
 'max_rtt': 5.7220458984375,
 'average_rtt': 2.3365020751953125}